# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking current working directory
print(os.getcwd())

# Get path to subfolder event data
filepath = os.getcwd() + '/event_data'

fnames = [os.path.join(filepath, f) for f in os.listdir(filepath) if 'csv' in f]

/Users/danieldiamond/Desktop/DEND_gdrive/DEND/project2_nosql_db_cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the fnames list 
for f in fnames:

    # reading csv file as dataframe
    df = pd.read_csv(f)
        
 # extracting each data row one by one and append it        
    for index, row in df.iterrows():
        full_data_rows_list.append(row.tolist()) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
df = pd.DataFrame(full_data_rows_list)[[0, 2, 3, 4, 5, 6, 7, 8, 12, 13, 16]]
df.columns = ['artist','firstName','gender','itemInSession','lastName','length',\
              'level','location','sessionId','song','userId']

# Drop any missing artists and save to csv file
df[df['artist'].notnull()].to_csv('event_datafile_new.csv', index=False)

In [4]:
# check the number of rows in your csv file
pd.read_csv('event_datafile_new.csv').shape[0]

6820

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [7]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [8]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifydb
    WITH REPLICATION =
    { 'class' : 'SimpleStrategy',
      'replication_factor' : 1 }"""
                   )
except Exception as e:
    print (e)

#### Set Keyspace

In [9]:
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print (e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

#### Query 1
Return the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
- Partition Key: sessionId
- Cluster Column: itemInSession
- Composite Primary Key: (sessionId, itemInSession)
- Additional Columns: artist, song, length

In [10]:
# Create Table
query = "CREATE TABLE IF NOT EXISTS session_library "
query = query + "(sessionId int, itemInSession int, \
                 artist varchar, song varchar, length float, \
                 PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print (e)

In [11]:
# Insert Data from CSV
filename = 'event_datafile_new.csv'

df = pd.read_csv(filename)

for index, row in df.iterrows():
    query = "INSERT INTO session_library \
             (sessionId, itemInSession, artist, song, length)"
    query = query + "VALUES (%s, %s, %s, %s, %s)"
    session.execute(query, (int(row.sessionId), int(row.itemInSession),
                            row.artist, row.song, float(row.length)))

In [12]:
# Select data accordingly to query 1

query = """SELECT artist, song, length from session_library 
            WHERE sessionId = 338 and itemInSession = 4 ;"""
try:
    rows = session.execute(query)
except Exception as e:
    print (e)
    
for row in rows:
    print (row.artist, row.song, row.length)

(u'Faithless', u'Music Matters (Mark Knight Dub)', 495.30731201171875)


In [13]:
# Drop Table
session.execute('DROP TABLE session_library');

### Query 2
Return only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
- Partition Key: userId
- Cluster Column: sessionId, itemInSession
- Composite Primary Key: (userId, sessionId, itemInSession)
- Additional Columns: firstName, lastName, artist, song

In [14]:
# Create Table
query = "CREATE TABLE IF NOT EXISTS song_playlist_session "
query = query + "(userId int, sessionId int, itemInSession int, \
                 firstName varchar, lastName varchar, \
                 artist varchar, song varchar, PRIMARY KEY \
                 (userId, sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print (e)

In [15]:
# Insert Data From Previous Dataframe (csv)
for index, row in df.iterrows():
    query = "INSERT INTO song_playlist_session \
             (userId, sessionId, itemInSession, \
             firstName, lastName, artist, song) "
    query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
    session.execute(query, (int(row.userId), int(row.sessionId), 
                            int(row.itemInSession), row.firstName, 
                            row.lastName, row.artist, row.song))

In [16]:
# Select data accordingly to query 2
query = """SELECT artist, song, firstName, lastName from song_playlist_session 
            WHERE userId = 10 and sessionId = 182 ;"""
try:
    rows = session.execute(query)
except Exception as e:
    print (e)
    
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)

(u'Down To The Bone', u"Keep On Keepin' On", u'Sylvie', u'Cruz')
(u'Three Drives', u'Greece 2000', u'Sylvie', u'Cruz')
(u'Sebastien Tellier', u'Kilometer', u'Sylvie', u'Cruz')
(u'Lonnie Gordon', u'Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', u'Sylvie', u'Cruz')


In [17]:
# Drop Table
session.execute('DROP TABLE song_playlist_session');

### Query 3
Return every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
- Partition Key: song
- Cluster Column: userId
- Composite Primary Key: (song, userId)
- Additional Columns: firstName, lastName

In [18]:
# Create Table
query = "CREATE TABLE IF NOT EXISTS user_songchoice_table "
query = query + "(song varchar, userId int, \
                 firstName varchar, lastName varchar, \
                 PRIMARY KEY (song, userId))"
try:
    session.execute(query)
except Exception as e:
    print (e)
    

In [19]:
# Insert Data From Previous Dataframe (csv)
for index, row in df.iterrows():
    query = "INSERT INTO user_songchoice_table \
             (song, userId, \
             firstName, lastName) "
    query = query + "VALUES (%s, %s, %s, %s)"
    session.execute(query, (row.song, int(row.userId), 
                            row.firstName, row.lastName))

In [20]:
# Select data according to query 3
query = """SELECT firstName, lastName from user_songchoice_table
         WHERE song = 'All Hands Against His Own';"""
try:
    rows = session.execute(query)
except Exception as e:
    print (e)
    
for row in rows:
    print (row.firstname, row.lastname)

(u'Jacqueline', u'Lynch')
(u'Tegan', u'Levine')
(u'Sara', u'Johnson')


In [21]:
# Drop Table
session.execute('DROP TABLE user_songchoice_table');

In [22]:
# Close session and cluster connection
session.shutdown()
cluster.shutdown()